<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5263bc38b217c1b0998f0ec3f42765899e0d46b2bb66ed8df0ef974a010e9d72
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-19 13:18:50
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: 40.31 K (0.29%)
Current PnL: -27.40 L (-17.7%)
CY Booked + Current PnL: -13.11 L (-8.47%)
-------------------
Total profit:  1.32 L
Total loss:  -28.73 L
-------------------
Total Booked + Current PnL: 13.87 L (10.85%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.23%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.04 L (65.91%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.9%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.54,-20.58,26.02,0.09,20827.0,-20736.0,80041.0,85.70,26.0,M-SC,2.30,253.0,-1.00,0.58,5.16,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-2.48,6.93,11.93,19.69,24461.0,13293.0,205035.0,-3.56,50.0,X-MC,2.50,78.0,0.54,1.48,14.20,XY25,NTT,AC
50,MASFIN,398.61,1.34,-3.03,25.86,22.05,24570.0,-2970.0,95010.0,-16.44,52.0,H-SC,6.40,164.0,-0.12,0.68,38.10,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.23,-16.08,19.18,0.02,28098.0,-28065.0,146494.0,123.21,42.0,M-SC,13.11,234.0,-1.00,1.05,27.42,OX40N,NTT,PAINTS
43,ITC,452.00,0.40,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,47.0,X-LC,2.13,5.0,-0.12,1.70,3.69,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.0,6.79,-22.84,71.11,32.03,72337.0,-30109.0,101726.0,-16.54,60.0,H-SC,9.57,148.0,-0.42,0.73,13.94,OX40N,NTT,IT
6,ASIANTILES,137.0,5.05,-4.32,89.88,81.67,80672.0,-4055.0,89755.0,8077.78,71.0,L-SC,13.01,271.0,-0.05,0.65,73.81,XR,NTT,CERAMICS
10,BANDHANBNK,400.0,2.86,-28.09,167.58,92.42,335691.0,-78242.0,200317.0,40.80,50.0,H-SC,9.25,170.0,-0.23,1.44,15.77,XY24,NTT,BANKS
64,SHALBY,327.0,2.83,-24.48,65.66,25.10,89817.0,-44352.0,136791.0,906.38,40.0,M-SC,11.00,250.0,-0.49,0.98,9.62,XY24,NTT,HEALTHCARE
57,RAJOOENG,143.1,2.46,-35.68,117.05,39.61,77171.0,-36570.0,65930.0,-56.41,37.0,H-SC,30.37,135.0,-0.47,0.47,3.88,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-2.88,-38.07,254.73,119.69,427152.0,-103073.0,167688.0,-37.58,41.0,H-SC,7.49,156.0,-0.24,1.21,32.47,AR,ATH,IT
82,VOLTAS,1530.00,-2.48,6.93,11.93,19.69,24461.0,13293.0,205035.0,-3.56,50.0,X-MC,2.50,78.0,0.54,1.48,14.20,XY25,NTT,AC
38,INDIAMART,4810.62,-1.78,-4.44,116.32,106.72,137096.0,-5475.0,117861.0,-52.73,37.0,H-SC,3.67,139.0,-0.04,0.85,17.90,AR,ATH,MISC
27,GLAXO,3466.20,-1.60,0.90,37.91,39.15,76226.0,1788.0,201072.0,-18.92,42.0,X-MC,10.51,60.0,0.02,1.45,30.16,X40,ATH,PHARMA
32,HCLTECH,1922.01,-1.41,6.30,17.34,24.73,44593.0,15242.0,257166.0,11.06,51.0,X-LC,6.89,7.0,0.34,1.85,22.50,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.9,-0.98,109.97,107.91,160773.0,-1447.0,146197.0,-19.34,59.0,M-SC,12.29,216.0,-0.01,1.05,9.01,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.90,-0.98,109.97,107.91,160773.0,-1447.0,146197.0,-19.34,59.0,M-SC,12.29,216.0,-0.01,1.05,9.01,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.23,-16.08,19.18,0.02,28098.0,-28065.0,146494.0,123.21,42.0,M-SC,13.11,234.0,-1.00,1.05,27.42,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.54,-20.58,26.02,0.09,20827.0,-20736.0,80041.0,85.70,26.0,M-SC,2.30,253.0,-1.00,0.58,5.16,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,6.79,-22.84,71.11,32.03,72337.0,-30109.0,101726.0,-16.54,60.0,H-SC,9.57,148.0,-0.42,0.73,13.94,OX40N,NTT,IT
66,SIS,528.00,0.61,-23.79,59.11,21.26,50020.0,-26410.0,84622.0,1996.89,52.0,H-SC,4.84,166.0,-0.53,0.61,14.33,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.1,8.33,59.02,72.27,108335.0,14108.0,183556.0,-6.99,69.0,M-LC,3.06,99.0,0.13,1.32,15.02,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.10,8.33,59.02,72.27,108335.0,14108.0,183556.0,-6.99,69.0,M-LC,3.06,99.0,0.13,1.32,15.02,XR,NTT,IT
25,FINCABLES,1641.55,0.90,-0.98,109.97,107.91,160773.0,-1447.0,146197.0,-19.34,59.0,M-SC,12.29,216.0,-0.01,1.05,9.01,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.78,-4.44,116.32,106.72,137096.0,-5475.0,117861.0,-52.73,37.0,H-SC,3.67,139.0,-0.04,0.85,17.90,AR,ATH,MISC
6,ASIANTILES,137.00,5.05,-4.32,89.88,81.67,80672.0,-4055.0,89755.0,8077.78,71.0,L-SC,13.01,271.0,-0.05,0.65,73.81,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,-0.01,-3.66,40.78,35.63,82337.0,-7668.0,201906.0,-17.18,36.0,H-MC,2.66,119.0,-0.09,1.45,12.40,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.09,0.14,-11.26,44.37,28.11,79300.0,-22685.0,178725.0,-35.73,25.0,X-MC,13.79,55.0,-0.29,1.29,0.14,X40,ATH,APPARELS
9,BAJAJHFL,181.50,0.16,-18.46,90.67,55.47,179868.0,-44910.0,198376.0,-27.82,27.0,X-MC,14.37,64.0,-0.25,1.43,1.64,X40N,ATH,FINANCE
8,AWL,485.00,0.25,-24.09,102.21,53.50,234119.0,-72694.0,229057.0,-63.25,29.0,X-MC,4.22,58.0,-0.31,1.65,1.74,XY24,NTT,FMCG
11,BATAINDIA,2096.00,0.23,-39.22,122.43,35.20,95754.0,-50459.0,78211.0,2.53,31.0,X-SC,16.62,92.0,-0.53,0.56,0.32,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.24,-26.07,122.05,64.15,214698.0,-62041.0,175910.0,-56.75,32.0,X-SC,3.20,82.0,-0.29,1.27,0.24,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.44,-1.86,26.84,24.48,66340.0,-4686.0,247170.0,-81.32,44.0,X-SC,0.78,86.0,-0.07,1.78,20.85,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.24,-4.22,26.48,21.14,57660.0,-9591.0,217748.0,-11.03,39.0,X-MC,1.34,74.0,-0.17,1.57,22.73,XY24,NTT,PAINTS
20,DABUR,735.0,-0.09,-3.79,49.36,43.70,117807.0,-9400.0,238668.0,-11.80,33.0,X-MC,2.02,72.0,-0.08,1.72,10.69,XY24,BTT,FMCG
43,ITC,452.0,0.40,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,47.0,X-LC,2.13,5.0,-0.12,1.70,3.69,X40,NTT,FMCG
78,UNITDSPR,1644.0,0.78,2.99,17.26,20.77,41621.0,7000.0,241144.0,-5.95,43.0,X-MC,2.25,62.0,0.17,1.74,10.11,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-0.05,-20.68,78.40,41.51,163778.0,-54465.0,208900.0,-5.42,35.0,X-MC,7.66,61.0,-0.33,1.50,0.08,XY25,ATH,FMCG
53,PAGEIND,51605.09,0.14,-11.26,44.37,28.11,79300.0,-22685.0,178725.0,-35.73,25.0,X-MC,13.79,55.0,-0.29,1.29,0.14,X40,ATH,APPARELS
3,ACC,3906.00,0.24,-26.07,122.05,64.15,214698.0,-62041.0,175910.0,-56.75,32.0,X-SC,3.20,82.0,-0.29,1.27,0.24,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,0.23,-39.22,122.43,35.20,95754.0,-50459.0,78211.0,2.53,31.0,X-SC,16.62,92.0,-0.53,0.56,0.32,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,-0.50,-48.10,193.49,52.31,145760.0,-69828.0,75332.0,-44.73,40.0,X-SC,7.31,91.0,-0.48,0.54,0.84,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,2.33,-29.08,101.43,42.84,46759.0,-18906.0,46100.0,-53.72,47.0,X-SC,34.75,83.0,-0.40,0.33,4.30,XY24,NTT,MISC
59,RELAXO,1176.00,-0.50,-48.10,193.49,52.31,145760.0,-69828.0,75332.0,-44.73,40.0,X-SC,7.31,91.0,-0.48,0.54,0.84,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.23,-39.22,122.43,35.20,95754.0,-50459.0,78211.0,2.53,31.0,X-SC,16.62,92.0,-0.53,0.56,0.32,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.73,-3.06,27.99,24.08,34446.0,-3884.0,123066.0,-10.94,40.0,X-SC,5.57,89.0,-0.11,0.89,15.96,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,1.09,-16.52,71.69,43.32,97470.0,-26912.0,135960.0,-28.31,40.0,X-SC,5.05,90.0,-0.28,0.98,4.39,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.46,-9.37,33.19,20.71,113770.0,-35453.0,342784.0,-22.01,70.0,X-LC,3.87,1.0,-0.31,2.47,14.54,X40,ATH,IT
41,INFY,1972.00,0.44,11.24,20.69,34.24,72010.0,35154.0,348042.0,-13.50,68.0,X-LC,4.87,2.0,0.49,2.50,20.45,X40,NTT,IT
75,TMPV,600.00,1.46,-28.09,71.01,22.98,115351.0,-63454.0,162444.0,-25.52,39.0,X-LC,6.84,3.0,-0.55,1.17,2.17,XY24,NTT,AUTO
81,VBL,671.64,-0.71,-5.01,42.83,35.67,128499.0,-15822.0,300020.0,-16.24,49.0,X-LC,3.94,4.0,-0.12,2.16,8.15,X40N,ATH,FMCG
43,ITC,452.00,0.40,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,47.0,X-LC,2.13,5.0,-0.12,1.70,3.69,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.49,-38.06,114.54,32.89,54776.0,-29384.0,47823.0,-690.94,51.0,L-MC,6.32,259.0,-0.54,0.34,31.70,XR,NTT,BANKS
6,ASIANTILES,137.00,5.05,-4.32,89.88,81.67,80672.0,-4055.0,89755.0,8077.78,71.0,L-SC,13.01,271.0,-0.05,0.65,73.81,XR,NTT,CERAMICS
50,MASFIN,398.61,1.34,-3.03,25.86,22.05,24570.0,-2970.0,95010.0,-16.44,52.0,H-SC,6.40,164.0,-0.12,0.68,38.10,XR,ATH,FINANCE
13,BSOFT,831.70,-0.72,-22.00,93.31,50.79,102374.0,-30939.0,109714.0,0.12,63.0,H-SC,6.86,151.0,-0.30,0.79,27.74,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.23,-16.08,19.18,0.02,28098.0,-28065.0,146494.0,123.21,42.0,M-SC,13.11,234.0,-1.00,1.05,27.42,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.7,-0.72,-22.00,93.31,50.79,102374.0,-30939.0,109714.0,0.12,63.0,H-SC,6.86,151.0,-0.30,0.79,27.74,XR,ATH,IT
84,WIPRO,420.0,0.10,8.33,59.02,72.27,108335.0,14108.0,183556.0,-6.99,69.0,M-LC,3.06,99.0,0.13,1.32,15.02,XR,NTT,IT
6,ASIANTILES,137.0,5.05,-4.32,89.88,81.67,80672.0,-4055.0,89755.0,8077.78,71.0,L-SC,13.01,271.0,-0.05,0.65,73.81,XR,NTT,CERAMICS
41,INFY,1972.0,0.44,11.24,20.69,34.24,72010.0,35154.0,348042.0,-13.50,68.0,X-LC,4.87,2.0,0.49,2.50,20.45,X40,NTT,IT
36,ICICIPRULI,790.0,0.64,7.53,21.58,30.73,45714.0,14830.0,211835.0,-17.59,61.0,X-MC,2.75,75.0,0.32,1.52,21.26,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.38
1,25,44.86
2,50,76.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.90
MC    30.85
LC    24.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.71
X40      23.14
X40N     12.98
XR        9.82
XY25      9.07
AR        8.96
OX40N     7.57
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.38
H-SC    23.72
X-LC    20.82
M-SC    11.62
X-SC     8.07
H-MC     4.79
L-SC     1.49
M-MC     1.34
M-LC     1.32
H-LC     1.11
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.30,-8.47,43.35
IT,13.17,-14.81,72.53
FINANCE,9.47,-21.01,73.64
MISC,7.26,-28.74,81.40
ELECTRICAL,5.98,-12.87,54.22
PAINTS,5.83,-14.09,31.04
INSURANCE,4.80,-1.55,36.30
PHARMA,4.12,-2.75,35.09
AUTO,2.85,-32.37,78.26


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3213242.0,21
AR,1325398.0,10
XR,1298108.0,13
X40,1006817.0,14
X40N,881869.0,9
OX40N,739407.0,10
XY25,380555.0,6
SR,279489.0,2
MH,79067.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3681731.0,25
M-SC,1443823.0,15
X-MC,1439672.0,16
X-LC,832904.0,11
X-SC,798905.0,8
H-MC,412804.0,3
L-SC,255335.0,3
M-LC,108335.0,1
H-LC,79421.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1292694.0      6
           AR         904327.0      5
M-SC       XY24       826058.0      6
H-SC       XR         789720.0      7
X-MC       X40        494461.0      7
           XY24       409586.0      3
           X40N       347386.0      4
H-SC       OX40N      336434.0      4
X-LC       X40        318924.0      5
M-SC       OX40N      316803.0      5
X-SC       X40N       309570.0      3
           XY24       295903.0      3
H-SC       SR         279489.0      2
X-LC       X40N       224913.0      2
H-MC       AR         216800.0      2
           XY24       196004.0      1
X-SC       X40        193432.0      2
X-LC       XY24       192997.0      2
X-MC       XY25       188239.0      2
M-SC       XR         176112.0      2
L-SC       XR         169165.0      2
M-SC       AR         124850.0      2
M-LC       XR         108335.0      1
X-LC       XY25        96070.0      2
L-SC       OX40N       86170.0      1
H-LC       AR          79421.0      1
H-SC       MH          79067.0      1
M-MC       XY25        59186.0      1
L-MC       XR          54776.0      1
L-LC       XY25        37060.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
